In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg
from scipy.io import loadmat

from dash import Dash, html, dcc, Input, Output, callback
import pandas as pd
import plotly.express as px

from turb.lst_scalar import lst_scalar

import plotly.graph_objects as go

In [2]:
mat_f = r'/Users/user/Documents/Projects/python_util/data/scalar_LST_matlab_linux.mat'
mat_data = loadmat(mat_f)
print(mat_data.keys())

dict_keys(['__header__', '__version__', '__globals__', 'A', 'B', 'D0p', 'D1p', 'D2p', 'D3p', 'D4p', 'E', 'M', 'N', 'Pr', 'R', 'Ra', 'Ri', 'T', 'Temperature', 'Tp', 'U', 'Up', 'Upp', 'V', 'aa', 'ans', 'bf', 'cf', 'ctl', 'dt', 'epsilon', 'fontsize_', 'None', 'ind', 'input', 'k2', 'kx', 'kz', 'linewidth_', 'omega', 'output', 'q', 'sp', 'tickfontsize_', 'tt', 'x', 'y_low', 'y_phys', 'y_top', '__function_workspace__'])


/opt/anaconda3/envs/turb/lib/python3.10/site-packages/scipy/io/matlab/_mio.py:227: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [3]:
df2 = pd.DataFrame(
   dict(
      # eigvals = mat_data['omega'][:, 0],
      eigvals_real = np.real(mat_data['omega'][:, 0]),
      eigvals_imag = np.imag(mat_data['omega'][:, 0]),
      eigvecs = [mat_data['Temperature'][:, ind] for ind in range(mat_data['omega'].shape[0])],
   )
)
df2["group"] = "solver"

result_y = mat_data["y_phys"][:, 0]

In [4]:
channel_lst = lst_scalar(384, 2)
# Temperature = channel_lst.D0 @ channel_lst.q[:,:]
Temperature = channel_lst.Temperature
df1 = pd.DataFrame(
   dict(
      # eigvals = channel_lst.omega,
      eigvals_real = np.real(channel_lst.omega),
      eigvals_imag = np.imag(channel_lst.omega),
      eigvecs = [Temperature[:, ind] for ind in range(channel_lst.omega.shape[0])],
   )
)
y_phys = channel_lst.y_phys[:, 0]

df1["group"] = "LST"

In [5]:
frames = [df1, df2]
df = pd.concat(frames, ignore_index=True)

In [6]:
marker_types = [dict(size=12, symbol="circle", ), dict(size=8, symbol="x", )]

In [11]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)


app.layout = html.Div([
    html.Div([

        html.Div([
            "Layer Number: ",
            dcc.Input(id='layer_number', value=384, type='number')
        ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    ]),
    html.Br(),

    html.Div([
        html.Div([
            html.Label('Display result'),
            dcc.Checklist(id="display_classes", options=df["group"].unique(),
                        value = []
            ),
        ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),

        
        html.Div(children=[
            html.Div(children=[
                "LST Index: ",
                dcc.Input(id='lst_index', value=0, type='number'),
                html.Div(id="group1_eigenvalues" )
            ], ),
            
            html.Div([
                "solver Index: ",
                dcc.Input(id='result_index', value=0, type='number'),
                html.Div(id="group2_eigenvalues" )
            ],),
        ], style={'width': '24%', 'display': 'inline-block'}),
    ]),
    
    html.Div([

        html.Div([
            dcc.Graph(
                id='eigenvals_plot',
                clickData={'points': [{'hovertext': 0, 'curveNumber' : 0}]}
            )
        ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),


        html.Div([
            dcc.Graph(id='eigenvector_plot',)
        ], style={'display': 'inline-block', 'width': '49%'})
    ]),
])

def create_eigvec_plot(y, eigvec, title):
    fig = px.scatter(x=y, y=eigvec[-y.shape[0]:])
    fig.update_traces(mode='lines+markers')

    fig.update_xaxes(showgrid=False)

    fig.add_annotation(x=0, y=0.85, xanchor='left', yanchor='bottom',
                       xref='paper', yref='paper', showarrow=False, align='left',
                       text=title)

    fig.update_layout(height=225, margin={'l': 20, 'b': 30, 'r': 10, 't': 10})
    
    return fig

@callback(
    Output('eigenvals_plot', 'figure'),
    Input('layer_number', 'value'),
    Input("display_classes", "value")
)
def update_graph(layer_number, display_class):
    # fig = px.scatter(df, x="eigvals_real", y = "eigvals_imag", color="group", symbol="group", hover_name=df.index)
    
    fig = go.Figure()

    # Add traces
    for ind, cls in enumerate(df["group"].unique()):
        if cls in display_class:
            fig.add_trace(go.Scatter(x=df[df["group"]==cls]["eigvals_real"], y = df[df["group"]==cls]["eigvals_imag"], hovertext=df[df["group"]==cls].index, opacity=0.5, marker=marker_types[ind],
                                mode='markers',
                                name=df["group"].unique()[ind]))
            
    return fig


@callback(
    Output('eigenvector_plot', 'figure'),
    Output('lst_index', 'value'),
    Output('result_index', 'value'),
    Output("group1_eigenvalues", "children"),
    Output("group2_eigenvalues", "children"),
    Input('eigenvals_plot', 'clickData'),
    Input('layer_number', 'value'),
    Input('lst_index', 'value'),
    Input('result_index', 'value'),
)
def update_eigenmode(clickData, layer_number, lst_ind, result_ind):
    norm_eigvec = lambda eigvecs: np.abs(eigvecs)/np.max(np.abs(eigvecs))
    
    
    print(clickData)
    eig_ind = clickData["points"][0]['hovertext']
    group = clickData["points"][0]["curveNumber"] # 0 - LST, 1 - result
    
    ind_set = [lst_ind, result_ind]
    if eig_ind in df[df["group"]==df["group"].unique()[0]].index:
        ind_set[0] = eig_ind
    if eig_ind in df[df["group"]==df["group"].unique()[1]].index:
        ind_set[1] = eig_ind


    # fig = px.line(x= y_phys, y= np.abs(df['eigvecs'][lst_ind]))
    mode_types = ["markers", "lines"]
    
    fig = go.Figure()

    # Add traces
    for ind, cls in enumerate(df["group"].unique()):
            fig.add_trace(go.Scatter(x=y_phys, y=norm_eigvec(np.abs(df['eigvecs'][ind_set[ind]])), marker=marker_types[ind],
                                mode=mode_types[ind],
                                name=df["group"].unique()[ind]))
        # fig.add_trace(go.Scatter(x=result_y, y=np.abs(df['eigvecs'][result_ind]),
        #                     mode='lines',
        #                     name=df["group"].unique()[1]))
    
    group1_eigvals = "%f + %f i" %(df['eigvals_real'][ind_set[0]], df['eigvals_imag'][ind_set[0]])
    group2_eigvals = "%f + %f i" %(df['eigvals_real'][ind_set[1]], df['eigvals_imag'][ind_set[1]])



    return fig, ind_set[0], ind_set[1], group1_eigvals, group2_eigvals


In [12]:
app.run(debug=True)

{'points': [{'hovertext': 0, 'curveNumber': 0}]}
